# Transfer Learning Task

The goal of this task is to illustrate how pretrained neural networks can be used for soving different classification problems.

## Data

We will use the [Cifar 10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html). It consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

Here are the classes in the dataset, as well as 10 random images from each:

![cifar10](https://github.com/mlcollege/rbi/blob/master/flagship/Deep-Learning-and-Image-Processing/images/cifar10.png?raw=1)



In [1]:
%tensorflow_version 2.x
from tensorflow.keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 1)


Prepare data

In [2]:
%tensorflow_version 2.x
from keras.utils import np_utils
import numpy as np

n_classes = 10

y_train = np_utils.to_categorical(y_train, n_classes)
y_test = np_utils.to_categorical(y_test, n_classes)
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

x_train_mean = np.mean(x_train, axis=0)
x_train -= x_train_mean
x_test -= x_train_mean

Import pretrained model

In [3]:
%tensorflow_version 2.x
from tensorflow.keras.applications import VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
for layer in base_model.layers[:-4]:
    layer.trainable = False

#from tensorflow.keras.applications.resnet50 import ResNet50
#base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

58900480/58889256 [==============================] - 1s 0us/step


Add new top layers

In [4]:
%tensorflow_version 2.x
from tensorflow.keras import models, layers
    
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))


Compile the model

In [5]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [6]:
model.fit(x_train, y_train,
          batch_size = 512, epochs = 20, verbose=1,
          validation_data=(x_test, y_test))

Epoch 1/20
98/98 [==============================] - 39s 256ms/step - loss: 1.4661 - accuracy: 0.4701 - val_loss: 0.9409 - val_accuracy: 0.6718
Epoch 2/20
98/98 [==============================] - 21s 212ms/step - loss: 0.8684 - accuracy: 0.7027 - val_loss: 0.8441 - val_accuracy: 0.7111
Epoch 3/20
98/98 [==============================] - 21s 211ms/step - loss: 0.7225 - accuracy: 0.7531 - val_loss: 0.7877 - val_accuracy: 0.7296
Epoch 4/20
98/98 [==============================] - 23s 236ms/step - loss: 0.6217 - accuracy: 0.7873 - val_loss: 0.7540 - val_accuracy: 0.7485
Epoch 5/20
98/98 [==============================] - 21s 210ms/step - loss: 0.5448 - accuracy: 0.8131 - val_loss: 0.7635 - val_accuracy: 0.7452
Epoch 6/20
98/98 [==============================] - 21s 210ms/step - loss: 0.4662 - accuracy: 0.8395 - val_loss: 0.8027 - val_accuracy: 0.7443
Epoch 7/20
98/98 [==============================] - 21s 210ms/step - loss: 0.4121 - accuracy: 0.8582 - val_loss: 0.8514 - val_accuracy: 0.7325

## Evaluate the model

First we need to convert probability vectors to class indices.

In [7]:
y_pred = model.predict(x_test)

print(y_pred.shape)

(10000, 10)


In [8]:
import numpy as np

y_test_class = np.argmax(y_test, axis=1)
y_pred_class = np.argmax(y_pred, axis=1)
print(y_pred_class.shape)

(10000,)


In [9]:
from sklearn import metrics
from sklearn.metrics import accuracy_score


print ("Test accuracy: {:.4f}".format(accuracy_score(y_test_class, y_pred_class)))
print ()
print(metrics.classification_report(y_test_class, y_pred_class, digits=4))

Test accuracy: 0.7439

              precision    recall  f1-score   support

           0     0.7817    0.8380    0.8089      1000
           1     0.8099    0.8350    0.8223      1000
           2     0.7040    0.6850    0.6944      1000
           3     0.5588    0.5230    0.5403      1000
           4     0.7423    0.6480    0.6919      1000
           5     0.6386    0.6290    0.6338      1000
           6     0.7366    0.8220    0.7769      1000
           7     0.8059    0.7970    0.8014      1000
           8     0.8940    0.8350    0.8635      1000
           9     0.7580    0.8270    0.7910      1000

    accuracy                         0.7439     10000
   macro avg     0.7430    0.7439    0.7424     10000
weighted avg     0.7430    0.7439    0.7424     10000



In [10]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test_class, y_pred_class))

[[838  23  29  10  18   7  11   9  34  21]
 [ 18 835   1   9   1   5   9   2  16 104]
 [ 56   9 685  61  57  39  61  17   2  13]
 [ 15  19  55 523  50 169  91  34  13  31]
 [ 17   6  84  53 648  50  55  68   8  11]
 [  8   7  39 175  26 629  44  51   3  18]
 [  5   9  40  54  26  21 822   2  10  11]
 [ 15   5  29  34  33  60   5 797   0  22]
 [ 65  34   5   8  11   2   5   2 835  33]
 [ 35  84   6   9   3   3  13   7  13 827]]
